In [1]:
import csv
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats

# To print data in tabular format
from tabulate import tabulate

In [2]:
#reading user-rating data file 
nfl_df = pd.read_csv('./data/nfl_games.csv', parse_dates = ['date'], index_col = ['date'])

#converting Order_Demand as int
nfl_df['result1']=nfl_df['result1'].astype(int)

In [3]:
display(nfl_df)

,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
date,,,,,,,,,,,
1920-09-26,1920,0,0,RII,STP,1503.947000,1300.000000,0.824651,48,0,1
1920-10-03,1920,0,0,AKR,WHE,1503.420000,1300.000000,0.824212,43,0,1
1920-10-03,1920,0,0,RCH,ABU,1503.420000,1300.000000,0.824212,10,0,1
1920-10-03,1920,0,0,DAY,COL,1493.002000,1504.908000,0.575819,14,0,1
1920-10-03,1920,0,0,RII,MUN,1516.108000,1478.004000,0.644171,45,0,1
...,...,...,...,...,...,...,...,...,...,...,...
2019-01-13,2018,0,1,NE,LAC,1640.171960,1647.624483,0.582068,41,28,1
2019-01-13,2018,0,1,NO,PHI,1669.105861,1633.114673,0.641378,20,14,1
2019-01-20,2018,0,1,NO,LAR,1682.450194,1648.424105,0.638772,23,26,0


The above dataframe contains the game results played b/w different teams having scores of each team and finally the result. '1' means team1 won that match and '0' means team2 won the match.

In [4]:
display(nfl_df.describe())

,season,neutral,playoff,elo1,elo2,elo_prob1,score1,score2,result1
count,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000,16274.000000
mean,1982.437569,0.005223,0.034779,1502.458394,1498.918374,0.584825,21.544058,18.578161,0.571034
std,25.448049,0.072084,0.183226,105.015371,104.541270,0.175310,11.289422,10.794566,0.494944
min,1920.000000,0.000000,0.000000,1119.595000,1156.551000,0.070953,0.000000,0.000000,0.000000
25%,1967.000000,0.000000,0.000000,1429.242750,1425.864750,0.461231,14.000000,10.000000,0.000000
50%,1987.000000,0.000000,0.000000,1504.015000,1500.185000,0.596354,21.000000,17.000000,1.000000
75%,2003.000000,0.000000,0.000000,1578.071500,1575.753000,0.719904,28.000000,26.000000,1.000000
max,2018.000000,1.000000,1.000000,1839.663000,1849.484000,0.970516,72.000000,73.000000,1.000000


In [5]:
display(nfl_df.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 16274 entries, 1920-09-26 to 2019-02-03
Data columns (total 11 columns):
season       16274 non-null int64
neutral      16274 non-null int64
playoff      16274 non-null int64
team1        16274 non-null object
team2        16274 non-null object
elo1         16274 non-null float64
elo2         16274 non-null float64
elo_prob1    16274 non-null float64
score1       16274 non-null int64
score2       16274 non-null int64
result1      16274 non-null int32
dtypes: float64(3), int32(1), int64(5), object(2)
memory usage: 1.4+ MB


None

In [6]:
# total number of games
total_games = len(nfl_df)

# total number of home wins
home_wins = 0
for i in nfl_df.result1:
    if i == 1:
        home_wins+=1

# total home wins/total number of games
home_win_rate = home_wins/total_games

print("Home Team Win Rate: {:.2f}% ".format(home_win_rate*100))

Home Team Win Rate: 57.10% 


In, above we are getting the Home Team Win rate as we can see from the above data result it is 57.10%.

In [7]:
HFA = 65.0     # Home field advantage is worth 65 Elo points
K = 20.0       # The speed at which Elo ratings change
REVERT = 1/3.0 # Between seasons, a team retains 2/3 of its previous season's rating

# Some between-season reversions of unknown origin
REVERSIONS = {'CBD1925': 1502.032, 
              'RAC1926': 1403.384, 
              'LOU1926': 1307.201, 
              'CIB1927': 1362.919, 
              'MNN1929': 1306.702,
              'BFF1929': 1331.943, 
              'LAR1944': 1373.977, 
              'PHI1944': 1497.988, 
              'ARI1945': 1353.939, 
              'PIT1945': 1353.939, 
              'CLE1999': 1300.0}


#### HFA = 65.0 :
Home field advantage is worth 65 Elo points.

#### The K-factor:
It regulates how quickly the ratings change in response to new information. A high K-factor tells Elo to be very sensitive to recent results, causing the ratings to jump around a lot based on each game’s outcome; a low K-factor makes Elo slow to change its opinion about teams, since every game carries comparatively little weight. In our NFL research, we found that the ideal K-factor for predicting future games is 20 — large enough that new results carry weight, but not so large that the ratings bounce around each week.

In [8]:
""" Initializes game objects from csv """
games = [item for item in csv.DictReader(open("./data/nfl_games.csv"))]

# Uncommenting these three lines will grab the latest game results for 2019, update team ratings accordingly, and make forecasts for upcoming games

games += [item for item in csv.DictReader(open('./data/nfl_games_2019.csv'))]

for game in games:
    game['season'], game['neutral'], game['playoff'] = int(game['season']), int(game['neutral']), int(game['playoff'])
    game['score1'], game['score2'] = int(game['score1']) if game['score1'] != '' else None, int(game['score2']) if game['score2'] != '' else None
    game['elo_prob1'], game['result1'] = float(game['elo_prob1']) if game['elo_prob1'] != '' else None, float(game['result1']) if game['result1'] != '' else None


In above code, reading the nfl_games CSV file and putting each data as dictionary.

Then, opening another CSV file which have the nfl_game list of 2019 which is not happened yet and I'm going to find out the winning probablity of the team.

In [9]:
header = games[0].keys()
rows=[]
counter =0
for x in games:
    rows.append(x.values())
    
    counter+=1
    if counter==20:
        break
        
        
print(tabulate(rows, header,tablefmt='simple'))

date          season    neutral    playoff  team1    team2       elo1     elo2    elo_prob1    score1    score2    result1
----------  --------  ---------  ---------  -------  -------  -------  -------  -----------  --------  --------  ---------
9-26-1920       1920          0          0  RII      STP      1503.95  1300        0.824651        48         0        1
10-3-1920       1920          0          0  AKR      WHE      1503.42  1300        0.824212        43         0        1
10-3-1920       1920          0          0  RCH      ABU      1503.42  1300        0.824212        10         0        1
10-3-1920       1920          0          0  DAY      COL      1493     1504.91     0.575819        14         0        1
10-3-1920       1920          0          0  RII      MUN      1516.11  1478        0.644171        45         0        1
10-3-1920       1920          0          0  CHI      MUT      1368.33  1300        0.682986        20         0        1
10-3-1920       1920        

Printing the above dataset using tabulate to print the data in good format.

In [10]:
# Initialize team objects to maintain ratings
teams = {}
for row in [item for item in csv.DictReader(open("./data/initial_elos.csv"))]:
    teams[row['team']] = {
        'name': row['team'],
        'season': None,
        'elo': float(row['elo'])
    }

In the above code, Opening other file having Elo rating of each team.  I'm initializing team objects to maintain ratings and by deafult season is None.

In [11]:
header = ['Name', 'Season', 'elo']
rows=[]
counter =0
for x in teams.items():
    rows.append([x[1]['name'], x[1]['season'],x[1]['elo']])      
        
print(tabulate(rows,header, tablefmt='grid'))

+--------+----------+---------+
| Name   | Season   |     elo |
+========+==========+=========+
| RII    |          | 1503.95 |
+--------+----------+---------+
| STP    |          | 1300    |
+--------+----------+---------+
| BFF    |          | 1478    |
+--------+----------+---------+
| WBU    |          | 1300    |
+--------+----------+---------+
| RCH    |          | 1503.42 |
+--------+----------+---------+
| ABU    |          | 1300    |
+--------+----------+---------+
| DAY    |          | 1493    |
+--------+----------+---------+
| COL    |          | 1504.91 |
+--------+----------+---------+
| MUN    |          | 1478    |
+--------+----------+---------+
| CHI    |          | 1368.33 |
+--------+----------+---------+
| MUT    |          | 1300    |
+--------+----------+---------+
| CBD    |          | 1504.69 |
+--------+----------+---------+
| PTQ    |          | 1300    |
+--------+----------+---------+
| AKR    |          | 1503.42 |
+--------+----------+---------+
| WHE   

Note: Season value is None that's why Season column is showing blank because I'm using tabulate to print the data.

In [12]:
for game in games:
    team1, team2 = teams[game['team1']], teams[game['team2']]

    # Revert teams at the start of seasons
    for team in [team1, team2]:
        if team['season'] and game['season'] != team['season']:
            k = "%s%s" % (team['name'], game['season'])
            if k in REVERSIONS:
                team['elo'] = REVERSIONS[k]
            else:
                team['elo'] = 1505.0*REVERT + team['elo']*(1-REVERT)
        team['season'] = game['season']
 

    # Elo difference includes home field advantage
    elo_diff = team1['elo'] - team2['elo'] + (0 if game['neutral'] == 1 else HFA)

    # This is the most important piece, where we set my_prob1 to our forecasted probability
    if game['elo_prob1'] != None:
        game['my_prob1'] = 1.0 / (math.pow(10.0, (-elo_diff/400.0)) + 1.0)

    # If game was played, maintain team Elo ratings
    if game['score1'] != None:

        # Margin of victory is used as a K multiplier
        pd = abs(game['score1'] - game['score2'])
        mult = math.log(max(pd, 1) + 1.0) * (2.2 / (1.0 if game['result1'] == 0.5 else ((elo_diff if game['result1'] == 1.0 else -elo_diff) * 0.001 + 2.2)))

        # Elo shift based on K and the margin of victory multiplier
        shift = (K * mult) * (game['result1'] - game['my_prob1'])

        # Apply shift
        team1['elo'] += shift
        team2['elo'] -= shift

In above code, for each games i'm finding out the winning probabilty. The winning probability is for team1 only. If probabilty is more than 50% than Team1 will have more winning chances else Team2 have more winning chances.

In [13]:

""" Evaluates and scores forecasts in the my_prob1 field against those in the elo_prob1 field for each game """
my_points_by_season, elo_points_by_season = {}, {}

forecasted_games = [g for g in games if g['result1'] != None]
upcoming_games = [g for g in games if g['result1'] == None and 'my_prob1' in g]

# Evaluate forecasts and group by season
for game in forecasted_games:

    # Skip unplayed games and ties
    if game['result1'] == None or game['result1'] == 0.5:
        continue

    if game['season'] not in elo_points_by_season:
        elo_points_by_season[game['season']] = 0.0
        my_points_by_season[game['season']] = 0.0

    # Calculate elo's points for game
    rounded_elo_prob = round(game['elo_prob1'], 2)
    elo_brier = (rounded_elo_prob - game['result1']) * (rounded_elo_prob - game['result1'])
    elo_points = 25 - (100 * elo_brier)
    elo_points = round(elo_points + 0.001 if elo_points < 0 else elo_points, 1) # Round half up
    if game['playoff'] == 1:
        elo_points *= 2
    elo_points_by_season[game['season']] += elo_points

    # Calculate my points for game
    rounded_my_prob = round(game['my_prob1'], 2)
    my_brier = (rounded_my_prob - game['result1']) * (rounded_my_prob - game['result1'])
    my_points = 25 - (100 * my_brier)
    my_points = round(my_points + 0.001 if my_points < 0 else my_points, 1) # Round half up
    if game['playoff'] == 1:
        my_points *= 2
    my_points_by_season[game['season']] += my_points

# Print individual seasons
for season in my_points_by_season:
    print("In %s, your forecasts would have gotten %s points. Elo got %s points." % (season, round(my_points_by_season[season], 2), round(elo_points_by_season[season], 2)))        
        

In 1920, your forecasts would have gotten 719.4 points. Elo got 719.4 points.
In 1921, your forecasts would have gotten 481.1 points. Elo got 481.1 points.
In 1922, your forecasts would have gotten 609.3 points. Elo got 609.3 points.
In 1923, your forecasts would have gotten 562.8 points. Elo got 562.8 points.
In 1924, your forecasts would have gotten 481.3 points. Elo got 481.3 points.
In 1925, your forecasts would have gotten 653.6 points. Elo got 653.6 points.
In 1926, your forecasts would have gotten 700.0 points. Elo got 700.0 points.
In 1927, your forecasts would have gotten 173.1 points. Elo got 173.1 points.
In 1928, your forecasts would have gotten 151.9 points. Elo got 151.9 points.
In 1929, your forecasts would have gotten 659.7 points. Elo got 659.7 points.
In 1930, your forecasts would have gotten 409.8 points. Elo got 409.8 points.
In 1931, your forecasts would have gotten 414.6 points. Elo got 414.6 points.
In 1932, your forecasts would have gotten 217.1 points. Elo got 

Evaluating result by season. 

For each year first I'm calculating actual elo's points for games. After that I'm finding my probability elo's point.
As we can see that actual and forecasted elo's points are the same.

In [14]:
# Show overall performance
my_avg = sum(my_points_by_season.values())/len(my_points_by_season.values())
elo_avg = sum(elo_points_by_season.values())/len(elo_points_by_season.values())
print("\nOn average, your forecasts would have gotten %s points per season. Elo got %s points per season.\n" % (round(my_avg, 2), round(elo_avg, 2)))



On average, your forecasts would have gotten 640.59 points per season. Elo got 640.57 points per season.



Now on average of all played games Elo got 640.59 points per season and the point which I've calcualted is also the same.

In [15]:
# Print forecasts for upcoming games
if len(upcoming_games) > 0:
    print("Forecasts for upcoming games:")
    print('\nDate\t\tTeam1 vs Team2\t\tWinning Proability')
    for game in upcoming_games:
        print("%s\t%s vs. %s\t\t\t%s%%" % (game['date'], game['team1'], game['team2'], int(round(100*game['elo_prob1']))))

        

Forecasts for upcoming games:

Date		Team1 vs Team2		Winning Proability
13-10-19	TB vs. CAR			40%
13-10-19	KC vs. HOU			71%
13-10-19	BAL vs. CIN			83%
13-10-19	CLE vs. SEA			43%
13-10-19	MIN vs. PHI			56%
13-10-19	MIA vs. WSH			57%
13-10-19	JAX vs. NO			35%
13-10-19	ARI vs. ATL			45%
13-10-19	LAR vs. SF			67%
13-10-19	NYJ vs. DAL			30%
13-10-19	DEN vs. TEN			48%
13-10-19	LAC vs. PIT			61%
14-10-19	GB vs. DET			62%
17-10-19	DEN vs. KC			31%
20-10-19	GB vs. OAK			70%
20-10-19	WSH vs. SF			35%
20-10-19	NYG vs. ARI			65%
20-10-19	BUF vs. MIA			79%
20-10-19	CIN vs. JAX			43%
20-10-19	ATL vs. LAR			40%
20-10-19	DET vs. MIN			52%
20-10-19	IND vs. HOU			63%
20-10-19	TEN vs. LAC			56%
20-10-19	CHI vs. NO			52%
20-10-19	SEA vs. BAL			62%
20-10-19	DAL vs. PHI			54%
21-10-19	NYJ vs. NE			14%
24-10-19	MIN vs. WSH			82%
27-10-19	DET vs. NYG			74%
27-10-19	IND vs. DEN			76%
27-10-19	CHI vs. LAC			65%
27-10-19	LAR vs. CIN			80%
27-10-19	BUF vs. PHI			49%
27-10-19	ATL vs. SEA			41%
27-10-19	TEN vs. TB	

In the above result I'm forecasting winning probality of matches which are going to happen in 2019.
For e.g. the first match which is going to happen b/w TB and CAR the winning probabilty of TB is 40%. i.e. CAR have higher chances to win the match.

---
#### Summary
I took three dataset(CSV files) from different sources. My aim was to group the the different dataset to get a meaningful result.

* First I took the NFL games file which had data from 1920 to 2018 and had different columns like season, team1 and team2 names, elo1, elo2, score1, score2, results, etc.

* Then I tried to figure out what is the winning rate of a Home Team and it was 57.10%.

* Then I had set some other variables like K-Factor to calculate the speed at which Elo ratings change and HFA (Home field advantage) and found out the winning probabilty.

* After that I took another CSV file which contained the list of games scheduled for 2019 with date and the two competing teams' names. For this data I have found out probability of the home team's win.

* Then I merged the two dataset and converted it into a Dictionary.

* After that, I took another CSV file which had the elo points of each teams.

* Then, for each game I found out the the winning probabilty of team1. If probabilty is more than 50% than Team1 will have more chances of winning, else Team2 will have more chances.

* For each year first I've calculated the actual elo's points for each game. After that I've found the probable elo's point. As we can see above from my results, the actual and forecasted elo's points are the same.

* Then I've calculated the average elo points per season which was 640.59.

* And finally I've forecasted winning probabilities of the teams participating in the games scheduled in 2019 which I got from the nfl_games_2019 csv file.